In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

FileNotFoundError: [Errno 2] File b'clean_data.csv' does not exist: b'clean_data.csv'

In [5]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier, RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [6]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [7]:
X_train, X_test, y_train, y_test = pdf_imdb['clean_text'], pdf_main['clean_text'], pdf_imdb['class'], pdf_main['class']

In [10]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import Adam

In [11]:
import os
embeddings_index = {}
f = open(os.path.join('../', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [12]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [13]:
EMBEDDING_DIM = 50
MAX_SEQUENCE_LENGTH = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [15]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [16]:
pad_train = pad_sequences(seq_train, maxlen=300)
pad_test = pad_sequences(seq_test, maxlen=300)

In [17]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2, return_sequences=True))
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2, return_sequences=True))
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2))
model.add(Dense(100, activation='relu'))
# model.add(LSTM(50, return_sequences=False, dropout=.2, recurrent_dropout=.2))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

W0725 22:48:06.529186 139731995322176 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 22:48:06.560404 139731995322176 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 22:48:06.610077 139731995322176 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0725 22:48:06.612203 139731995322176 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please us

In [18]:
model.fit(pad_train, y_enh, epochs=2, batch_size=64, validation_data=(pad_test, y_test))

Train on 51340 samples, validate on 660 samples
Epoch 1/2
51340/51340 [==============================] - 2109s 41ms/step - loss: 0.4669 - acc: 0.7806 - val_loss: 0.3744 - val_acc: 0.8470
Epoch 2/2
51340/51340 [==============================] - 1708s 33ms/step - loss: 0.2963 - acc: 0.8764 - val_loss: 0.3034 - val_acc: 0.8727


In [19]:
model.save('deep_rnn.hdf5')

In [20]:
accuracy_score(y_test, model.predict_classes(pad_test))

0.8727272727272727